<a href="https://colab.research.google.com/github/doukansurel/Retrieval-Augmented-Generation/blob/main/RAG_7_01_24.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Kurulum

In [ ]:
!pip install llama-index
!pip install pypdf
!pip install rank-bm25
!pip install sentence-transformers

In [204]:
import requests
import os
from google.colab import drive
import pandas as pd
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [205]:
os.environ["OPENAI_API_KEY"] = "<OPENAI_API_KEY>"

#Belgeleri Oluşturma

 * Bu kısımda ben SimpleDirectoryReader kullandım.Yüklenecek belge formatına göre değişiklik gösterebilir.
  

In [206]:
import warnings
import logging

warnings.filterwarnings("ignore")
logger = logging.getLogger("pypdf")
logger.setLevel(logging.ERROR)

In [207]:
from llama_index import SimpleDirectoryReader

reader = SimpleDirectoryReader(
    input_files=["/content/drive/MyDrive/RAG/Belgeler/risk-gruplarinin-belirlenmesine-ve-kredi-sinirlarina-iliskin-yonetmelik-1-1.pdf"]
)

documents = reader.load_data()

In [208]:
from llama_index import Document

doc_text = "\n\n".join([d.get_content() for d in documents])
docs = [Document(text=doc_text)]

#Düğüm Oluşturma

In [209]:

from llama_index.node_parser import SentenceSplitter
from llama_index.schema import IndexNode

node_parser = SentenceSplitter(chunk_size=1024)
base_nodes = node_parser.get_nodes_from_documents(docs)
for idx,node in enumerate(base_nodes):
  node.id_ = f"node-{idx}"

#Gömme Modeli Oluşturma ve Modelin Yüklenmesi

In [210]:

from llama_index.embeddings import resolve_embed_model

embed_model = resolve_embed_model("local:BAAI/bge-small-en")
llm = OpenAI(model="gpt-3.5-turbo")
service_context = ServiceContext.from_defaults(
    llm=llm, embed_model=embed_model
)

In [211]:
print(f"Yüklenen belge {len(documents)} sayfadan oluşmaktadır. ")

Yüklenen belge 14 sayfadan oluşmaktadır. 


In [212]:
from llama_index import OpenAIEmbedding, PromptHelper

In [214]:
prompt_helper = PromptHelper(

  context_window=4096,

  num_output=256  ,

  chunk_overlap_ratio=0.1,

  chunk_size_limit=None

)

In [215]:
from llama_index.llms import OpenAI

llm = OpenAI(model="gpt-3.5-turbo")

In [216]:
from llama_index import ServiceContext

service_context = ServiceContext.from_defaults(

  llm=llm,

  embed_model=embed_model,

  node_parser=node_parser,

  prompt_helper=prompt_helper

)

In [217]:
from llama_index import VectorStoreIndex
from llama_index.storage.docstore import SimpleDocumentStore
from llama_index.storage.index_store import SimpleIndexStore
from llama_index.vector_stores import SimpleVectorStore
from llama_index.node_parser import SentenceSplitter
from llama_index.storage import StorageContext

storage_context = StorageContext.from_defaults(
    docstore=SimpleDocumentStore(),
    vector_store=SimpleVectorStore(),
    index_store=SimpleIndexStore(),
)
storage_context.docstore.add_documents(base_nodes)


# Meta Veri Referansları: Özetler + Daha büyük bir parçaya atıfta bulunan Oluşturulmuş Sorular

In [218]:

import nest_asyncio
from llama_index.node_parser import SentenceSplitter
from llama_index.schema import IndexNode
from llama_index.extractors import (
    SummaryExtractor,
    QuestionsAnsweredExtractor,
)
nest_asyncio.apply()

In [219]:
extractors = [
    SummaryExtractor(summaries=["self"], show_progress=True),
    QuestionsAnsweredExtractor(questions=5, show_progress=True),
]

In [220]:

node_to_metadata = {}
for extractor in extractors:
    metadata_dicts = extractor.extract(base_nodes)
    for node, metadata in zip(base_nodes, metadata_dicts):
        if node.node_id not in node_to_metadata:
            node_to_metadata[node.node_id] = metadata
        else:
            node_to_metadata[node.node_id].update(metadata)

100%|██████████| 20/20 [00:15<00:00,  1.28it/s]


In [221]:
# cache metadata dicts
def save_metadata_dicts(path, data):
    with open(path, "w") as fp:
        json.dump(data, fp)


def load_metadata_dicts(path):
    with open(path, "r") as fp:
        data = json.load(fp)
    return data

In [222]:

import copy

all_nodes = copy.deepcopy(base_nodes)
for node_id, metadata in node_to_metadata.items():
    for val in metadata.values():
        all_nodes.append(IndexNode(text=val, index_id=node_id))

In [223]:

all_nodes_dict = {n.node_id: n for n in all_nodes}


In [224]:
vector_index_metadata = VectorStoreIndex(
    all_nodes, service_context=service_context
)

In [ ]:
vector_index_metadata.storage_context.persist(persist_dir="./storage1")

In [225]:
vector_retriever_metadata = vector_index_metadata.as_retriever(
    similarity_top_k=2
)

In [226]:
from llama_index.retrievers import RecursiveRetriever

retriever_metadata = RecursiveRetriever(
    "vector",
    retriever_dict={"vector": vector_retriever_metadata},
    node_dict=all_nodes_dict,
    verbose=False,
)


In [227]:
from llama_index.response.notebook_utils import display_source_node

nodes = retriever_metadata.retrieve(
"Kredi sınırları hesaplamalarında kredi riski azaltım tekniklerinin kullanılmasına ilişkin esaslar neleri kapsar?")
for node in nodes:
    display_source_node(node, source_length=2000)


**Node ID:** node-15<br>**Similarity:** 0.9116180463756253<br>**Text:** (4)  Aralarında  aynı  risk  grubunda  değerlendirilmelerini  gerektiren  başka  bir  ilişki  bulunmaması  kaydıyla
sadece  takas  işlemleri  sebebiyle  birden  fazla  merkezi  karşı  tarafın  aynı  risk  grubunda  dikkate  alınmaları
zorunlu değildir.
Kredi sınırları hesaplamalarında kredi riski azaltım tekniklerinin kullanılmasına ilişkin esaslar 
MADDE 12 
(1) Kredi sınırları hesaplamalarında, 6/9/2014 tarihli ve 29111 sayılı Resmî Gazete’de yayımlanan Kredi Riski
Azaltım  Tekniklerine  İlişkin  Tebliğ  uyarınca  asgari  şartları  sağlayan  finansal  teminatlar,  fonlanmamış  kredi
korumaları ve bilanço içi netleştirme risk tutarı azaltımı olarak dikkate alınır. Münhasıran kredi riskine esas
tutarın içsel derecelendirmeye dayalı yaklaşımlar ile hesaplanmasında dikkate alınan teminatlar kredi sınırları
hesaplamasında risk tutarı azaltımında kullanılmaz.
(2) Birinci fıkrada belirtilen kredi riski azaltım tekniklerinin, Bankaların Sermaye Yeterliliğinin Ölçülmesine ve
Değerlendirilmesine  İlişkin  Yönetmelik  uyarınca  yapılan  sermaye  yeterliliği  hesaplamalarında  kullanılması
durumunda, kredi sınırları hesaplamalarında da kullanılması zorunludur.
(3) Finansal teminatların kullanılması durumunda;
a) Kredi Riski Azaltım Tekniklerine İlişkin Tebliğde belirtilen Basit Finansal Teminat Yönteminin kullanıldığı
durumda, kredi tutarının finansal teminatın piyasa değeri kadar olan kısmı,
b) Karşı taraf kredi riski hesaplamasında dikkate alınan teminat tutarı,
c)  Kredi  Riski  Azaltım  Tekniklerine  İlişkin  Tebliğde  belirtilen  Kapsamlı  Finansal  Teminat  Yönteminin
kullanıldığı  durumda,  anılan  Tebliğin  42  nci  maddesinde  belirtilen  Standart  Volatilite  Ayarlaması  Yaklaşımı
kullanılarak hesaplanan teminatın, aynı Tebliğin 40 ıncı maddesinde tanımlanan volatilite ayarlı değeri,
koruma  sağlanan  tarafın  risk  tutarından  indirilir  ve  indirilen  bu  tutar  koruma  sağlayıcısının  kredi  sınırları
hesaplamasına dâhil edilir.
(4)  Fonlanmamış  kredi  koruma...<br>

**Node ID:** node-16<br>**Similarity:** 0.9052521006679854<br>**Text:** (5)  Bilanço  içi  netleştirmeye  ilişkin  Kredi  Riski  Azaltım  Tekniklerine  İlişkin  Tebliğde  belirtilen  şartların
sağlanması halinde, söz konusu netleştirme sözleşmeleri kredi sınırları hesaplamasında Kredi Riski Azaltım
Tekniklerine İlişkin Tebliğde belirtildiği şekilde dikkate alınır.
(6) Kredi korumasının kalan vadesinin korunan alacağın kalan vadesinden az olması durumunda oluşan vade
uyumsuzluğu halinde;
a) Korumanın başlangıç vadesinin bir yıldan kısa olması,
Risk Gruplarının Belirlenmesine ve Kredi Sınırlarına İlişkin Yönetmelik
Sayfa 11/14


b) Korumanın kalan vadesinin üç aydan kısa olması,
durumlarından herhangi birinin varlığı halinde ilgili kredi koruması dikkate alınmaz.
(7) Vade uyumsuzluğu durumunda kredi sınırları hesaplamasında dikkate alınacak kredi koruması tutarının,
Bankaların Sermaye Yeterliliğinin Ölçülmesine ve Değerlendirilmesine İlişkin Yönetmelik uyarınca sermaye
yeterliliği hesaplamalarında kullanılan aynı yaklaşım ile belirlenmesi gerekir.
Merkezi yurt dışında kurulu bankaların Türkiye’deki şubelerinin merkezleri ve yurt dışında kurulu diğer şubeler 
ile yaptıkları işlemler 
MADDE 13 
(1) Merkezi yurt dışında kurulu bankaların Türkiye’deki şubelerinin, merkezlerine ve yurt dışında kurulu diğer
şubelere  yaptıkları  tevdiat,  Kanunun  48  inci  maddesinin  birinci  fıkrası  kapsamında  kredi  olarak  sayılır  ve
bankanın dâhil olduğu risk grubunun risk tutarları hesaplamasına dâhil edilir.
(2) Merkez ve yurt dışında kurulu diğer şubelerden sağlanan fonlar 12 nci madde uygulamasında finansal
teminat kapsamında nakit teminat olarak değerlendirilir.
Kredi sınırlarına tabi olmayan işlemler 
MADDE 14 
(1) Aşağıdaki işlemler kredi sınırlarına tabi tutulmaz:
a)  Merkezi  yönetimlerden,  merkez  bankalarından  veya  Bankaların  Sermaye  Yeterliliğinin  Ölçülmesine  ve
Değerlendirilmesine  İlişkin  Yönetmelik  uyarınca  merkezi  yönetimlerden  alacaklarla  aynı  uygulamaya  tâbi
tutulan kamu kuruluşlarından alacaklar ile Kredi Riski Azal...<br>

In [228]:
from llama_index.query_engine import RetrieverQueryEngine

query_engine = RetrieverQueryEngine.from_args(
    retriever=retriever_metadata,
    service_context=service_context,
)


In [229]:
response = query_engine.query(
"Kredi sınırları hesaplamalarında kredi riski azaltım tekniklerinin kullanılmasına ilişkin esaslar neleri kapsar?")

In [230]:
from llama_index.response.notebook_utils import display_response

display_response(response)

**`Final Response:`** Kredi sınırları hesaplamalarında kredi riski azaltım tekniklerinin kullanılmasına ilişkin esaslar, finansal teminatlar, fonlanmamış kredi korumaları ve bilanço içi netleştirme risk tutarı azaltımını içerir. Bu tekniklerin kullanılabilmesi için belirli şartların sağlanması gerekmektedir. Ayrıca, bu tekniklerin sermaye yeterliliği hesaplamalarında kullanılması durumunda kredi sınırları hesaplamalarında da kullanılması zorunludur.

In [231]:
score=[]
content=[]
for node in response.source_nodes:
  score.append(node.get_score())
  content.append(node.get_content())

In [232]:
import pandas as pd

df = pd.DataFrame({"Score":score,"Content":content})
df["Response"] = response

In [233]:
display(df)

,Score,Content,Response
0,0.911618,"(4) Aralarında aynı risk grubunda değerlendirilmelerini gerektiren başka bir ilişki bulunmaması kaydıyla\nsadece takas işlemleri sebebiyle birden fazla merkezi karşı tarafın aynı risk grubunda dikkate alınmaları\nzorunlu değildir.\nKredi sınırları hesaplamalarında kredi riski azaltım tekniklerinin kullanılmasına ilişkin esaslar \nMADDE 12 \n(1) Kredi sınırları hesaplamalarında, 6/9/2014 tarihli ve 29111 sayılı Resmî Gazete’de yayımlanan Kredi Riski\nAzaltım Tekniklerine İlişkin Tebliğ uyarınca asgari şartları sağlayan finansal teminatlar, fonlanmamış kredi\nkorumaları ve bilanço içi netleştirme risk tutarı azaltımı olarak dikkate alınır. Münhasıran kredi riskine esas\ntutarın içsel derecelendirmeye dayalı yaklaşımlar ile hesaplanmasında dikkate alınan teminatlar kredi sınırları\nhesaplamasında risk tutarı azaltımında kullanılmaz.\n(2) Birinci fıkrada belirtilen kredi riski azaltım tekniklerinin, Bankaların Sermaye Yeterliliğinin Ölçülmesine ve\nDeğerlendirilmesine İlişkin Yönetmelik uyarınca yapılan sermaye yeterliliği hesaplamalarında kullanılması\ndurumunda, kredi sınırları hesaplamalarında da kullanılması zorunludur.\n(3) Finansal teminatların kullanılması durumunda;\na) Kredi Riski Azaltım Tekniklerine İlişkin Tebliğde belirtilen Basit Finansal Teminat Yönteminin kullanıldığı\ndurumda, kredi tutarının finansal teminatın piyasa değeri kadar olan kısmı,\nb) Karşı taraf kredi riski hesaplamasında dikkate alınan teminat tutarı,\nc) Kredi Riski Azaltım Tekniklerine İlişkin Tebliğde belirtilen Kapsamlı Finansal Teminat Yönteminin\nkullanıldığı durumda, anılan Tebliğin 42 nci maddesinde belirtilen Standart Volatilite Ayarlaması Yaklaşımı\nkullanılarak hesaplanan teminatın, aynı Tebliğin 40 ıncı maddesinde tanımlanan volatilite ayarlı değeri,\nkoruma sağlanan tarafın risk tutarından indirilir ve indirilen bu tutar koruma sağlayıcısının kredi sınırları\nhesaplamasına dâhil edilir.\n(4) Fonlanmamış kredi korumalarının kullanılması durumunda, koruma sağlanan tutar koruma sağlanan\ntarafın risk tutarından indirilir ve indirilen bu tutar koruma sağlayıcısının kredi sınırları hesaplamasına dâhil\nedilir.\n(5) Bilanço içi netleştirmeye ilişkin Kredi Riski Azaltım Tekniklerine İlişkin Tebliğde belirtilen şartların\nsağlanması halinde, söz konusu netleştirme sözleşmeleri kredi sınırları hesaplamasında Kredi Riski Azaltım\nTekniklerine İlişkin Tebliğde belirtildiği şekilde dikkate alınır.","Kredi sınırları hesaplamalarında kredi riski azaltım tekniklerinin kullanılmasına ilişkin esaslar, finansal teminatlar, fonlanmamış kredi korumaları ve bilanço içi netleştirme risk tutarı azaltımını içerir. Bu tekniklerin kullanılabilmesi için belirli şartların sağlanması gerekmektedir. Ayrıca, bu tekniklerin sermaye yeterliliği hesaplamalarında kullanılması durumunda kredi sınırları hesaplamalarında da kullanılması zorunludur."
1,0.905252,"(5) Bilanço içi netleştirmeye ilişkin Kredi Riski Azaltım Tekniklerine İlişkin Tebliğde belirtilen şartların\nsağlanması halinde, söz konusu netleştirme sözleşmeleri kredi sınırları hesaplamasında Kredi Riski Azaltım\nTekniklerine İlişkin Tebliğde belirtildiği şekilde dikkate alınır.\n(6) Kredi korumasının kalan vadesinin korunan alacağın kalan vadesinden az olması durumunda oluşan vade\nuyumsuzluğu halinde;\na) Korumanın başlangıç vadesinin bir yıldan kısa olması,\nRisk Gruplarının Belirlenmesine ve Kredi Sınırlarına İlişkin Yönetmelik\nSayfa 11/14\n\n\nb) Korumanın kalan vadesinin üç aydan kısa olması,\ndurumlarından herhangi birinin varlığı halinde ilgili kredi koruması dikkate alınmaz.\n(7) Vade uyumsuzluğu durumunda kredi sınırları hesaplamasında dikkate alınacak kredi koruması tutarının,\nBankaların Sermaye Yeterliliğinin Ölçülmesine ve Değerlendirilmesine İlişkin Yönetmelik uyarınca sermaye\nyeterliliği hesaplamalarında kullanılan aynı yaklaşım ile belirlenmesi gerekir.\nMerkezi yurt dışında kurulu bankaların Türkiye’deki şubelerinin merkezleri ve yurt dışında kurulu diğer şu

#Evaluation

# Mean Reciprocal Rank (MRR):

MRR, her sorgu için sistemin doğruluğunu, en üst sıradaki ilgili belgenin sıralamasına bakarak değerlendirir. Spesifik olarak, bu, tüm sorgulardaki bu sıralamaların karşılıklarının ortalamasıdır.

In [234]:

from llama_index.evaluation import (
    generate_question_context_pairs,
    EmbeddingQAFinetuneDataset,
)
from llama_index.llms import OpenAI

import nest_asyncio

nest_asyncio.apply()

In [235]:
eval_dataset = generate_question_context_pairs(
    base_nodes, OpenAI(model="gpt-3.5-turbo")
)

100%|██████████| 20/20 [00:32<00:00,  1.62s/it]


In [236]:
eval_dataset.save_json("/content/data/dataset.json")


In [237]:
import pandas as pd
from llama_index.evaluation import RetrieverEvaluator, get_retrieval_results_df

# set vector retriever similarity top k to higher
top_k = 10


def display_results(names, results_arr):
    """Display results from evaluate."""

    hit_rates = []
    mrrs = []
    for name, eval_results in zip(names, results_arr):
        metric_dicts = []
        for eval_result in eval_results:
            metric_dict = eval_result.metric_vals_dict
            metric_dicts.append(metric_dict)
        results_df = pd.DataFrame(metric_dicts)

        hit_rate = results_df["hit_rate"].mean()
        mrr = results_df["mrr"].mean()
        hit_rates.append(hit_rate)
        mrrs.append(mrr)

    final_df = pd.DataFrame(
        {"retrievers": names, "hit_rate": hit_rates, "mrr": mrrs}
    )
    display(final_df)

In [238]:
vector_retriever_metadata = vector_index_metadata.as_retriever(
    similarity_top_k=top_k
)
retriever_metadata = RecursiveRetriever(
    "vector",
    retriever_dict={"vector": vector_retriever_metadata},
    node_dict=all_nodes_dict,
    verbose=True,
)
retriever_evaluator = RetrieverEvaluator.from_metric_names(
    ["mrr", "hit_rate"], retriever=retriever_metadata
)
# try it out on an entire dataset
results_metadata = await retriever_evaluator.aevaluate_dataset(
    eval_dataset, show_progress=True
)

  0%|          | 0/48 [00:00<?, ?it/s]

Retrieving with query id None: d) Based on the market value of the partnership's assets
Retrieved node with id, entering: node-8
Retrieving with query id node-8: d) Based on the market value of the partnership's assets
Retrieved node with id, entering: node-12
Retrieving with query id node-12: d) Based on the market value of the partnership's assets
Retrieved node with id, entering: node-2
Retrieving with query id node-2: d) Based on the market value of the partnership's assets
Retrieved node with id, entering: node-17
Retrieving with query id node-17: d) Based on the market value of the partnership's assets
Retrieving text node: b)  Bankaların,  merkezi  yurt  dışında  kurulu  bankalar  dâhil,  diğer  bankalar  ile  gerçekleştirdikleri  işlemlerden
kaynaklanan gün içerisindeki alacaklar.
c)  Bankaların  Sermaye  Yeterliliğinin  Ölçülmesine  ve  Değerlendirilmesine  İlişkin  Yönetmelik  kapsamında
nitelikli işlemleri gerçekleştiren merkezi karşı taraflarla yapılan takas işlemlerinden k

100%|██████████| 48/48 [00:07<00:00,  6.22it/s]

Retrieved node with id, entering: node-8
Retrieving with query id node-8: How is the risk amount calculated for transactions with central counterparties according to the regulation?
Retrieved node with id, entering: node-10
Retrieving with query id node-10: How is the risk amount calculated for transactions with central counterparties according to the regulation?
Retrieved node with id, entering: node-13
Retrieving with query id node-13: How is the risk amount calculated for transactions with central counterparties according to the regulation?
Retrieved node with id, entering: node-17
Retrieving with query id node-17: How is the risk amount calculated for transactions with central counterparties according to the regulation?
Retrieved node with id, entering: node-11
Retrieving with query id node-11: How is the risk amount calculated for transactions with central counterparties according to the regulation?
Retrieving with query id None: How is the risk amount for non-performing loans and

In [239]:

results_df = get_retrieval_results_df(
    [

        "Retriever (Metadata References)"
    ],
    [results_metadata],
)
display(results_df)

,retrievers,hit_rate,mrr
0,Retriever (Metadata References),1.0,0.84375


#Yanıt Değerlendirmesi

In [240]:
from llama_index.evaluation import FaithfulnessEvaluator
evaluator = FaithfulnessEvaluator(service_context=service_context)

response_eval = query_engine.query(
"Kredi sınırları hesaplamalarında kredi riski azaltım tekniklerinin kullanılmasına ilişkin esaslar neleri kapsar?")
eval_result = evaluator.evaluate_response(response=response_eval)
print(str(eval_result.passing))

True


In [242]:
from llama_index.evaluation import generate_question_context_pairs
from llama_index.evaluation import RetrieverEvaluator
qa_dataset = generate_question_context_pairs(
    base_nodes,
    llm=llm,
    num_questions_per_chunk=2
)

100%|██████████| 20/20 [00:31<00:00,  1.59s/it]


In [243]:
queries = list(qa_dataset.queries.values())

In [244]:
from llama_index.evaluation import FaithfulnessEvaluator
faithfulness = FaithfulnessEvaluator(service_context=service_context)

eval_query = queries[10]
eval_query

'What are the criteria used to determine the economic dependency between customers in the evaluation of risk groups according to the regulations mentioned in the document? Provide examples for each criterion.'

In [245]:
response_vector = query_engine.query(eval_query)

In [246]:
eval_result = faithfulness.evaluate_response(response=response_vector)

# Uygunluk Değerlendiricisi

In [247]:
from llama_index.evaluation import RelevancyEvaluator

relevancy = RelevancyEvaluator(service_context=service_context)

In [249]:
query_str="Kredi sınırları hesaplamalarında kredi riski azaltım tekniklerinin kullanılmasına ilişkin esaslar neleri kapsar?"


In [250]:
response_vector = query_engine.query(query_str)

# Relevancy evaluation
relevancy_eval_result = relevancy.evaluate_response(
    query=query_str, response=response_vector
)

In [251]:
evalution_name = ["Faitfullnes","Relevancy"]
r_eval= str(relevancy_eval_result.passing)
f_eval = str(eval_result.passing)
result=[]
result.append(f_eval)
result.append(r_eval)


response_result = pd.DataFrame(
    {"Evalution Name":evalution_name,"Passing":result}
)

In [252]:
display(response_result)

,Evalution Name,Passing
0,Faitfullnes,True
1,Relevancy,True
